In [1]:
import torch
import torch.nn as nn
import math

In [2]:
config = {
    "vocab_size": 14,                # V: 5 special + 9 emojis
    "hidden_size": 8,                # d_model: The main vector size
    "max_position_embeddings": 16,   # max_len: Longest possible sequence
    "type_vocab_size": 2,            # 0 for Sentence A, 1 for Sentence B
    "hidden_dropout_prob": 0.1       # Standard BERT dropout
}

TOKEN_TO_ID = {
    '[PAD]': 0,
    '[UNK]': 1,
    '[CLS]': 2,
    '[SEP]': 3,
    '[MASK]': 4,
    '☀️': 5,
    '🌙': 6,
    '🧑': 7,
    '☕': 8,
    '🥐': 9,
    '💻': 10,
    '🏃': 11,
    '📖': 12,
    '😴': 13,
}

ID_TO_TOKEN = {id: token for token, id in TOKEN_TO_ID.items()}
TOKEN_TO_TEXT = {
    '[PAD]': '[PAD]',
    '[UNK]': '[UNK]',
    '[CLS]': '[CLS]',
    '[SEP]': '[SEP]',
    '[MASK]': '[MASK]',
    '☀️': 'day',
    '🌙': 'night',
    '🧑': 'person',
    '☕': 'coffee',
    '🥐': 'food',
    '💻':'work',
    '🏃': 'run',
    '📖': 'book',
    '😴': 'sleep',
}